##### ✅ 코루틴 사용하기
Kotlin 코루틴에서 자주 사용하는 기능들을 정리한 내용입니다.

---

#### 1. runBlocking
- 코루틴을 블로킹 방식으로 실행
- 테스트나 진입 지점에서 주로 사용

```kotlin
runBlocking {
    println("Start")
    delay(1000)
    println("End")
}
```

---

#### 2. launch / async / withContext
| 함수 | 설명 |
|------|------|
| launch | 결과가 없는 비동기 작업 수행 |
| async | 결과가 필요한 비동기 작업 수행 (await()로 값 반환) |
| withContext | 특정 Dispatcher로 context 전환 |

---

#### 3. Flow & 연산자
| 함수 | 설명 |
|------|------|
| flow | 콜드 스트림 생성 |
| collect | 스트림의 값을 수집 |
| collectLatest | 가장 최신 값만 처리, 이전 값은 취소됨 |
| map / filter | 값 변환 / 조건 필터링 |
| flatMapLatest / switchMap | 가장 최신 flow만 유지 |
| debounce | 일정 시간 간격 내 연속 입력 무시 |
| retry / catch | 재시도 및 예외 처리 |

---

#### 4. Android에서 자주 쓰는 CoroutineScope
| 스코프 | 설명 |
|--------|------|
| viewModelScope | ViewModel 내 생명주기와 연결된 코루틴 실행 |
| lifecycleScope | Activity/Fragment 생명주기 내 안전한 실행 |
| repeatOnLifecycle | 특정 상태(Lifecycle.State.STARTED 등)에서만 수집 수행 |

---

#### 5. Subject & StateFlow
| 타입 | 설명 |

RxJava의 Subject와 Kotlin Coroutines의 StateFlow/SharedFlow는 모두 데이터 스트림을 여러 곳에서 관찰하고 발행할 수 있게 해주는 도구입니다. 각각의 특징과 사용 시점을 이해하는 것이 중요합니다.

##### RxJava Subject 종류

| Subject 타입      | 설명 |
|-------------------|------|
| **PublishSubject**   | 구독 이후에 발행된 데이터만 Observer에게 전달. 구독 전 데이터는 전달되지 않음. |
| **BehaviorSubject**  | 구독 시점에 가장 최근 데이터 1개를 즉시 전달, 이후 발행 데이터도 전달. |
| **ReplaySubject**    | 구독 시점에 과거 N개(혹은 모든) 데이터를 버퍼에 저장 후 모두 전달, 이후 데이터도 전달. |
| **AsyncSubject**     | 스트림이 완료(onComplete)될 때 마지막으로 발행된 데이터 1개만 전달. |

##### Coroutines의 StateFlow & SharedFlow

| 타입 | 특징 | 사용 예시 |
|------|------|----------|
| **StateFlow** | 항상 최신 상태 1개를 보존. 구독 시 현재 값 즉시 전달. 상태(State) 관리에 적합. | UI 상태, 단일 상태 데이터 |
| **SharedFlow** | 여러 값 이벤트를 브로드캐스트. replay 버퍼 크기 지정 가능. 이벤트성 데이터에 적합. | 알림, 일회성 이벤트, 메시지 브로드캐스트 |

> **StateFlow vs SharedFlow**
> - StateFlow: 상태를 나타냄(항상 값이 존재), 구독 시 즉시 현재 값 전달.
> - SharedFlow: 이벤트를 나타냄(값이 없을 수도 있음), replay로 과거 이벤트 일부 전달 가능, 기본적으로는 구독 시 값 없음.

##### 언제 어떤 타입을 써야 할까?
- **PublishSubject / SharedFlow**: 구독 이후에만 이벤트를 받고 싶을 때, 이벤트성 데이터(예: 클릭 이벤트, 알림 등).
- **BehaviorSubject / StateFlow**: 상태를 항상 유지하고, 구독 시 최신 상태를 받고 싶을 때(예: UI 상태, 설정 값 등).
- **ReplaySubject / SharedFlow(replay > 0)**: 과거 여러 이벤트를 새 구독자에게 전달하고 싶을 때.
- **AsyncSubject**: 오직 마지막 데이터만 필요하고, 완료 시점에만 전달할 때.

##### RxJava Subject 예제
```java
// PublishSubject 예시
PublishSubject<String> subject = PublishSubject.create();
subject.onNext("A"); // 구독 전 데이터
subject.subscribe(System.out::println); // 이후 데이터만 받음
subject.onNext("B"); // "B"만 출력됨

// BehaviorSubject 예시
BehaviorSubject<Integer> behavior = BehaviorSubject.createDefault(0);
behavior.onNext(1);
behavior.subscribe(System.out::println); // "1" 즉시 출력
behavior.onNext(2); // "2" 출력

// ReplaySubject 예시
ReplaySubject<String> replay = ReplaySubject.createWithSize(2);
replay.onNext("X");
replay.onNext("Y");
replay.onNext("Z");
replay.subscribe(System.out::println); // "Y", "Z" 출력

// AsyncSubject 예시
AsyncSubject<Integer> async = AsyncSubject.create();
async.onNext(10);
async.onNext(20);
async.onComplete();
async.subscribe(System.out::println); // "20"만 출력
```

##### Kotlin StateFlow & SharedFlow 예제
```kotlin
// StateFlow: 항상 값이 존재, 상태 보존
val stateFlow = MutableStateFlow("Initial")
CoroutineScope(Dispatchers.Main).launch {
    stateFlow.collect { println("State: $it") }
}
stateFlow.value = "Updated"

// SharedFlow: 이벤트성 데이터, replay 활용 가능
val sharedFlow = MutableSharedFlow<String>(replay = 2)
CoroutineScope(Dispatchers.Main).launch {
    sharedFlow.collect { println("Shared: $it") }
}
sharedFlow.emit("Event1")
sharedFlow.emit("Event2")
sharedFlow.emit("Event3") // 새 구독자는 Event2, Event3을 받음 (replay=2)
```

#### 📌 실무 예시 1 - 사용자 입력 처리 (검색창)
```kotlin
val queryFlow = MutableStateFlow("")

val resultsFlow = queryFlow
    .debounce(300)
    .filter { it.isNotBlank() }
    .flatMapLatest { keyword -> searchApi(keyword) }

lifecycleScope.launch {
    resultsFlow.collectLatest { results ->
        updateUI(results)
    }
}
```

---

# #### 📌 실무 예시 2 - ViewModel에서 비동기 API 호출
```kotlin
fun fetchData() {
    viewModelScope.launch {
        val result = withContext(Dispatchers.IO) {
            repository.loadData()
        }
        _uiState.value = result
    }
}
```

---

#### 📌 실무 예시 3 - suspend 함수 정의 및 사용

```kotlin
suspend fun fetchUser(): User {
    delay(500)
    return User("kim", 30)
}

fun example() {
    CoroutineScope(Dispatchers.IO).launch {
        val user = fetchUser()
        println("user=$user")
    }
}
```

---

#### 📌 실무 예시 4 - flowOf를 활용한 데이터 스트림

```kotlin
val numbers = flowOf(1, 2, 3, 4, 5)

runBlocking {
    numbers.collect { println(it) }
}
```

---

#### 📌 실무 예시 5 - channel을 사용한 코루틴 간 통신

```kotlin
val channel = Channel<Int>()

CoroutineScope(Dispatchers.Default).launch {
    repeat(5) {
        channel.send(it)
    }
    channel.close()
}

CoroutineScope(Dispatchers.Default).launch {
    for (value in channel) {
        println("Received: $value")
    }
}
```

---

#### 📌 실무 예시 6 - SharedFlow와 StateFlow

```kotlin
val sharedFlow = MutableSharedFlow<String>()
val stateFlow = MutableStateFlow("Initial")

// 수집
CoroutineScope(Dispatchers.Main).launch {
    sharedFlow.collect { println("Shared: $it") }
    stateFlow.collect { println("State: $it") }
}

// 발행
CoroutineScope(Dispatchers.IO).launch {
    sharedFlow.emit("Shared message")
    stateFlow.value = "New state"
}
```

In [48]:
%use coroutines
@file:DependsOn("io.reactivex.rxjava3:rxjava:3.1.8")
@file:DependsOn("io.reactivex.rxjava3:rxkotlin:3.0.1")

In [49]:
runBlocking {
    println("Hello, World!")
    delay(1000L)
    println("Goodbye, World!1")
}

Hello, World!
Goodbye, World!1


In [50]:
// flatMapLatest 사용해보기
// flatMapLatest 사용하여 flow를 변환하는 예제
// upstream 과 downstream으로 나눠서 고민
// switchMap 같음.
// -----1-----------------2-----------------3------------------
// -----A-----B-----C-----A-----B-----C-----A-----B-----C
// 1->A
// 1->B
// 1->C
// 2->A
// 2->B
// 2->C
// 3->A
// 3->B
// 3->C
runBlocking {
    val upstream = flowOf(1, 2, 3)
    val downstream = flowOf("A", "B", "C")

    val result = upstream.flatMapLatest { value ->
        downstream.map { "$value -> $it" }
    }

    result.collect { println(it) }
}

1 -> A
1 -> B
1 -> C
2 -> A
2 -> B
2 -> C
3 -> A
3 -> B
3 -> C


In [51]:
import java.util.concurrent.TimeUnit

val subject = PublishSubject.create<String>()

subject.onNext("Hello1") // 구독 전 데이터는 전달되지 않음

subject.subscribe { println("PublishSubject 1: $it") } // 구독 후 발행된 데이터만 받음

subject.onNext("Hello2")
subject.onNext("Hello2")
subject.onNext("Hello2")
subject.onNext("Hello2") // 이후 발행된 데이터는 모두 전달됨

PublishSubject 1: Hello2
PublishSubject 1: Hello2
PublishSubject 1: Hello2
PublishSubject 1: Hello2


In [52]:
import io.reactivex.rxjava3.subjects.BehaviorSubject
import java.util.concurrent.TimeUnit

val subject = BehaviorSubject.create<String>()

subject.onNext("Hello1")
subject.onNext("Hello1")
subject.onNext("Hello1")
subject.onNext("Hello1") // 구독 전 1개만 전달

subject.subscribe { println("BehaviorSubject 1: $it") } // 구독 시점에 가장 최근 데이터 1개를 즉시 전달

subject.onNext("Hello2")
subject.onNext("Hello2")
subject.onNext("Hello2")
subject.onNext("Hello2")// 이후는 무조건


BehaviorSubject 1: Hello1
BehaviorSubject 1: Hello2
BehaviorSubject 1: Hello2
BehaviorSubject 1: Hello2
BehaviorSubject 1: Hello2


In [53]:
import io.reactivex.rxjava3.subjects.BehaviorSubject
import io.reactivex.rxjava3.subjects.ReplaySubject
import java.util.concurrent.TimeUnit

val subject = ReplaySubject.create<String>()

subject.onNext("Hello1")
subject.onNext("Hello1")
subject.onNext("Hello1")
subject.onNext("Hello1")

subject.subscribe { println("ReplaySubject 1: $it") } // 구독 시점에 과거 N개(혹은 모든) 데이터를 버퍼에 저장 후 모두 전달

subject.onNext("Hello2")
subject.onNext("Hello2")
subject.onNext("Hello2")
subject.onNext("Hello2")


ReplaySubject 1: Hello1
ReplaySubject 1: Hello1
ReplaySubject 1: Hello1
ReplaySubject 1: Hello1
ReplaySubject 1: Hello2
ReplaySubject 1: Hello2
ReplaySubject 1: Hello2
ReplaySubject 1: Hello2
